In [1]:
# Imports 
using ReverseDiff: JacobianTape, jacobian!, compile
using Interpolations: LinearInterpolation, Line
using CSV
using Revise
using NPZ
using DataFrames
include("../src/noise_src_jl/get_noise.jl")
include("../src/noise_src_jl/get_interpolation_functions.jl")

get_a_weighting_interpolation_functions (generic function with 1 method)

In [6]:
# Inputs 
struct Data
    supp_af_freq
    supp_af_angles
    supp_af
    aw_freq
    aw_db
    abs_alt
    abs_freq
    abs
    supp_fi_angles
    supp_fi_freq
    supp_fi
    supp_fd_angles
    supp_fd_freq
    supp_fd
    Faddeeva_itau_re
    Faddeeva_itau_im
    Faddeeva_real
    Faddeeva_imag
    jet_D_angles
    jet_D_velocity
    jet_D
    jet_xi_angles
    jet_xi_velocity
    jet_xi
    jet_F_angles
    jet_F_temperature
    jet_F_velocity
    jet_F_strouhal
    jet_F
    noy_spl
    noy_freq
    noy
end
df_hsr = Matrix(DataFrame(CSV.File("../data/sources/airframe/hsr_suppression.csv")))
df_aw = Matrix(DataFrame(CSV.File("../data/levels/weighting_a.csv")))
df_abs = Matrix(DataFrame(CSV.File("../data/isa/atmospheric_absorption.csv")))
df_i = Matrix(DataFrame(CSV.File("../data/sources/fan/liner_inlet_suppression.csv")))
df_d = Matrix(DataFrame(CSV.File("../data/sources/fan/liner_discharge_suppression.csv")))
df_re = Matrix(DataFrame(CSV.File("../data/propagation/Faddeeva_real_small.csv")))
df_im = Matrix(DataFrame(CSV.File("../data/propagation/Faddeeva_imag_small.csv")))
df_D = Matrix(DataFrame(CSV.File("../data/sources/jet/directivity_function.csv")))
df_xi = Matrix(DataFrame(CSV.File("../data/sources/jet/strouhal_correction.csv")))
df_F = npzread("../data/sources/jet/spectral_function_extended_T.npy")
jet_a = [0, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180]
jet_t = [0, 1, 2, 2.5, 3, 3.5, 4, 5, 6, 7]
jet_v = [-0.4, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.4]
jet_s = [-2, -1.6, -1.3, -1.15, -1, -0.824, -0.699, -0.602, -0.5, -0.398, -0.301, -0.222, 0, 0.477, 1, 1.6, 1.7, 2.5]
df_noy = Matrix(DataFrame(CSV.File("../data/levels/spl_noy.csv")))

f = [50.11,63.09,79.43,100.,125.89,158.49,199.53,251.19,316.23,398.11,501.19,630.96,794.33,1000.,1258.93,1584.89,1995.26,2511.89,3162.28,3981.07,5011.87,6309.57,7943.28,10000.]
f_sb = [   45.69436719,    47.8554422 ,    50.11872336,    52.48904442,   54.97146773,    57.52580003,    60.24643228,    63.09573445,   66.07979186,    69.20497765,    72.42069149,    75.84576457,   79.43282347,    83.18952918,    87.12390499,    91.17224886,   95.48416039,   100.        ,   104.72941228,   109.68249797,  114.77906094,   120.20743594,   125.89254118,   131.84651848,  138.08208392,   144.49827655,   151.33219579,   158.48931925,  165.98493258,   173.83504436,   181.91255231,   190.5159469 ,  199.5262315 ,   208.9626496 ,   218.84535481,   229.01433483,  239.84536691,   251.18864315,   263.06838969,   275.50997842,  288.31196578,   301.9474273 ,   316.22776602,   331.18348082,  346.84651304,   362.96326025,   380.12928926,   398.10717055,  416.93529997,   436.65388926,   456.94367188,   478.55442202,  501.18723363,   524.89044421,   549.71467735,   575.25800028,  602.4643228 ,   630.95734448,   660.79791862,   692.04977655,  724.20691489,   758.45764568,   794.32823472,   831.89529182,  871.23904992,   911.72248856,   954.84160391,  1000.        , 1047.29412282,  1096.82497969,  1147.79060935,  1202.0743594 , 1258.92541179,  1318.46518484,  1380.82083923,  1444.98276553, 1513.32195792,  1584.89319246,  1659.84932576,  1738.35044364, 1819.12552313,  1905.15946905,  1995.26231497,  2089.62649595, 2188.4535481 ,  2290.14334831,  2398.4536691 ,  2511.88643151, 2630.68389691,  2755.09978424,  2883.11965784,  3019.47427305, 3162.27766017,  3311.83480822,  3468.4651304 ,  3629.6326025 , 3801.2928926 ,  3981.07170553,  4169.35299973,  4366.53889259, 4569.43671876,  4785.54422016,  5011.87233627,  5248.90444211, 5497.14677346,  5752.58000284,  6024.64322803,  6309.5734448 , 6607.97918625,  6920.49776548,  7242.06914895,  7584.57645675, 7943.28234724,  8318.95291817,  8712.39049922,  9117.22488558, 9548.4160391 , 10000.        , 10472.94122821, 10968.24979695]

data = Data(df_hsr[2:end, 1], df_hsr[1,2:end], df_hsr[2:end,2:end], df_aw[:,1], df_aw[:,2], 
    df_abs[2:end, 1], df_abs[1,2:end], df_abs[2:end,2:end], df_i[1, 2:end], df_i[2:end, 1], 
    df_i[2:end, 2:end], df_d[1, 2:end], df_d[2:end, 1], df_d[2:end, 2:end], df_re[1, 2:end], 
    df_re[2:end, 1], df_re[2:end,2:end], df_im[2:end,2:end], df_D[1, 2:end], df_D[2:end, 1], 
    df_D[2:end, 2:end], df_xi[1, 2:end], df_xi[2:end, 1], df_xi[2:end, 2:end], jet_a, jet_t, 
    jet_v, jet_s, df_F, df_noy[2:end, 1], df_noy[1, 2:end], df_noy[2:end, 2:end])

settings = Dict()
settings["r_0"] = 0.3048
settings["A_e"] = 10.669
settings["p_ref"] = 2e-5
settings["core_jet_suppression"] = false
settings["case_name"] = "stca"
settings["n_frequency_bands"] = 24
settings["n_frequency_subbands"] = 5
settings["direct_propagation"] = true
settings["absorption"] = true
settings["ground_effects"] = true
settings["lateral_attenuation"] = true
settings["core_jet_suppression"] = false
settings["case_name"] = "stca"
settings["epnl_dt"] = 0.5
settings["fan_BB_method"] = "geae"
settings["fan_RS_method"] = "allied_signal"
settings["fan_id"] = false
settings["fan_igv"] = false
settings["n_harmonics"] = 10
settings["fan_ge_flight_cleanup"] = "takeoff"
settings["fan_combination_tones"] = false
settings["fan_liner_suppression"] = false
settings["shielding"] = true
settings["ground_resistance"] = 291.0 * 515.379
settings["incoherence_constant"] = 0.01
settings["n_shock"] = 8
settings["lateral_attenuation_engine_mounting"] = "underwing"
settings["tones_under_800Hz"] = false
settings["fan_inlet_source"] = false
settings["fan_discharge_source"] = false
settings["core_source"] = false
settings["jet_mixing_source"] = true
settings["jet_shock_source"] = false
settings["airframe_source"] = false
settings["atmosphere_type"] = "stratified"
settings["x_observer_array"] = reshape(vcat(1000:450:6000, 450*ones(12), 1.2*ones(12)), (12,3))
settings["levels_int_metric"] = "epnl"

struct PynaInterpolations
    f_hsr_supp
    f_aw
    f_abs
    f_D_core
    f_S_core
    f_supp_fi
    f_supp_fd
    f_F3IB
    f_F3DB
    f_F3TI
    f_F3TD
    f_F2CT
    f_TCS_takeoff_ih1
    f_TCS_takeoff_ih2
    f_TCS_approach_ih1
    f_TCS_approach_ih2
    f_faddeeva_real
    f_faddeeva_imag
    f_omega_jet
    f_log10P_jet
    f_log10D_jet
    f_xi_jet
    f_log10F_jet
    f_m_theta_jet
    f_noy
end
f_hsr_supp = get_airframe_interpolation_functions(data)
f_aw = get_a_weighting_interpolation_functions(data)
f_abs = get_atmospheric_absorption_interpolation_functions(data)
f_D_core, f_S_core = get_core_interpolation_functions()
f_supp_fi, f_supp_fd, f_F3IB, f_F3DB, f_F3TI, f_F3TD, f_F2CT, f_TCS_takeoff_ih1, f_TCS_takeoff_ih2, f_TCS_approach_ih1, f_TCS_approach_ih2 = get_fan_interpolation_functions(settings, data)
f_faddeeva_real, f_faddeeva_imag = get_ground_effects_interpolation_functions(data)
f_omega_jet, f_log10P_jet, f_log10D_jet, f_xi_jet, f_log10F_jet, f_m_theta_jet = get_jet_mixing_interpolation_functions(data)
f_noy = get_noy_interpolation_functions(data)

pyna_ip = PynaInterpolations(f_hsr_supp, 
    f_aw, 
    f_abs, 
    f_D_core, 
    f_S_core, 
    f_supp_fi, 
    f_supp_fd,
    f_F3IB,
    f_F3DB,
    f_F3TI,
    f_F3TD,
    f_F2CT, 
    f_TCS_takeoff_ih1, 
    f_TCS_takeoff_ih2, 
    f_TCS_approach_ih1, 
    f_TCS_approach_ih2,
    f_faddeeva_real, 
    f_faddeeva_imag, 
    f_omega_jet, 
    f_log10P_jet, 
    f_log10D_jet, 
    f_xi_jet, 
    f_log10F_jet, 
    f_m_theta_jet, 
    f_noy)

struct Aircraft
    mtow
    n_eng
    comp_lst
    af_S_h
    af_S_v
    af_S_w
    af_b_f
    af_b_h
    af_b_v
    af_b_w
    af_S_f
    af_s
    af_d_mg
    af_d_ng
    af_l_mg
    af_l_ng
    af_n_mg
    af_n_ng
    af_N_mg
    af_N_ng
    c_d_g
    mu_r
    B_fan
    V_fan
    RSS_fan
    M_d_fan
    inc_F_n
    TS_lower
    TS_upper
    af_clean_w
    af_clean_h
    af_clean_v
    af_delta_wing
end
af = Aircraft(55000.0,3,["wing", "tail_h", "les", "tef", "lg"],   20.16,21.3677,150.41, 6.096,5.6388,4.7244,20.51304,11.1484,1.0,0.9144,0.82296,2.286,1.8288,4.0,  2.0,  2.0,  1.0,0.0240,  0.0175,25,  48,  300.0,1.68,0.25, 0.65,  1.0,  true,false, true,  true)

shield = ones(24)

sealevel_atmosphere = Dict()
sealevel_atmosphere["g"] = 9.80665
sealevel_atmosphere["R"] = 287.05
sealevel_atmosphere["T_0"] = 288.15
sealevel_atmosphere["c_0"] = 340.294
sealevel_atmosphere["p_0"] = 101325.
sealevel_atmosphere["rho_0"] = 1.225
sealevel_atmosphere["mu_0"] = 1.7894e-5
sealevel_atmosphere["k_0"] = 25.5e-3
sealevel_atmosphere["lapse_0"] = 0.0065
sealevel_atmosphere["gamma"] = 1.4
sealevel_atmosphere["rh_0"] = 70.
sealevel_atmosphere["I_0"] = 409.74

409.74

In [7]:
# Input vector 
n_t = 5
n_1 = 101
n_2 = 105

x = [0.00000000e+00, 3.32251665e-01, 8.91600837e-01, 1.80841957e+00, 3.25092060e+00, 4.96324890e+00, 7.11635610e+00, 9.70586070e+00, 1.25640032e+01, 1.59506637e+01, 1.98012627e+01, 2.37970660e+01, 2.84773081e+01, 3.34742444e+01, 3.87562806e+01, 4.47163398e+01, 5.09753616e+01, 5.75157600e+01, 6.46873992e+01, 7.23768936e+01, 8.01319200e+01, 8.86907040e+01, 9.75586628e+01, 1.06743829e+02, 1.16607157e+02, 1.26836245e+02, 1.37378811e+02, 1.48674968e+02, 1.60121997e+02, 1.72254428e+02, 1.84776831e+02, 1.97864943e+02, 2.11323308e+02, 2.25168916e+02, 2.39910375e+02, 2.54619102e+02, 2.70130524e+02, 2.86143497e+02, 3.02682250e+02, 3.19630264e+02, 3.36992282e+02, 3.55172204e+02, 3.73955722e+02, 3.92619787e+02, 4.12435552e+02, 4.32703010e+02, 4.53274153e+02, 4.74794067e+02, 4.96445278e+02, 5.19005098e+02, 5.41834599e+02, 5.65701500e+02, 5.89610544e+02, 6.14504560e+02, 6.39929147e+02, 6.65892246e+02, 6.92325973e+02, 7.19643321e+02, 7.47621060e+02, 7.76159865e+02, 8.05274742e+02, 8.34906506e+02, 8.65447618e+02, 8.96696056e+02, 9.28579653e+02, 9.60877867e+02, 9.94278892e+02, 1.02811609e+03, 1.06282672e+03, 1.09811542e+03, 1.13449537e+03, 1.17142290e+03, 1.20908285e+03, 1.24731604e+03, 1.28660881e+03, 1.32654065e+03, 1.33018469e+03, 1.34861290e+03, 1.36704111e+03, 1.38563126e+03, 1.40435390e+03, 1.42309017e+03, 1.44208531e+03, 1.46108044e+03, 1.48022377e+03, 1.49948835e+03, 1.51876628e+03, 1.53829786e+03, 1.55782945e+03, 1.57749648e+03, 1.59727435e+03, 1.61706383e+03, 1.63707413e+03, 1.65708442e+03, 1.67114274e+03, 1.69135752e+03, 1.71157229e+03, 1.73191246e+03, 1.75229444e+03, 1.77270080e+03, 1.79318031e+03, 1.81365982e+03, 1.83420334e+03, 1.85476820e+03, 1.87535058e+03, 1.89598554e+03, 1.91662050e+03, 1.93729432e+03, 1.95798110e+03, 1.97864535e+03, 1.99924203e+03, 2.00130169e+03, 2.10497530e+03, 2.20902266e+03, 2.31318059e+03, 2.41729270e+03, 2.52126017e+03, 2.62487324e+03, 2.72799844e+03, 2.83072720e+03, 2.93333424e+03, 3.03594128e+03, 3.13862461e+03, 3.24138026e+03, 3.34424269e+03, 3.44718501e+03, 3.55025701e+03, 3.65341269e+03, 3.75669805e+03, 3.86005952e+03, 3.96351242e+03, 4.06707968e+03, 4.17074594e+03, 4.27453034e+03, 4.37843666e+03, 4.48246112e+03, 4.58658837e+03, 4.69085288e+03, 4.79522397e+03, 4.89973232e+03, 5.00434346e+03, 5.10910785e+03, 5.18159408e+03, 5.28658546e+03, 5.39189722e+03, 5.49763010e+03, 5.60372761e+03, 5.71009833e+03, 5.81662705e+03, 5.92333529e+03, 6.03016656e+03, 6.13709040e+03, 6.24410456e+03, 6.35118080e+03, 6.45829318e+03, 6.49993402e+03, 6.60709184e+03, 6.71435096e+03, 6.82164696e+03, 6.92902160e+03, 7.03643632e+03, 7.14390560e+03, 7.25141176e+03, 7.35897248e+03, 7.46657008e+03, 7.57422224e+03, 7.68190808e+03, 7.78962760e+03, 7.89740488e+03, 8.00523672e+03, 8.11308456e+03, 8.22100464e+03, 8.32894072e+03, 8.43693136e+03, 8.54497976e+03, 8.65306184e+03, 8.76117760e+03, 8.86934792e+03, 8.97755192e+03, 9.08595210e+03, 9.14385886e+03, 9.25229822e+03, 9.36062692e+03, 9.46901102e+03, 9.57742838e+03, 9.68587900e+03, 9.79438502e+03, 9.90292708e+03, 1.00115245e+04, 1.01201553e+04, 1.02288192e+04, 1.03375386e+04, 1.04462913e+04, 1.05550772e+04, 1.06639185e+04, 1.07727930e+04, 1.08817008e+04, 1.09906641e+04, 1.10996633e+04, 1.12087152e+04, 1.13177782e+04, 1.14268966e+04, 1.15360732e+04, 1.16452830e+04, 1.17545233e+04, 1.18637969e+04, 1.19731287e+04, 1.20825158e+04, 1.21503393e+04]
y = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
z = [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.09600000e-03, 3.04800000e-02, 5.48640000e-02, 2.20980000e-01, 4.34340000e-01, 7.18566000e-01, 1.21539000e+00, 1.71221400e+00, 2.39420400e+00, 3.13791600e+00, 3.93496800e+00, 4.89204000e+00, 5.84911200e+00, 6.95020200e+00, 8.09929800e+00, 9.30636696e+00, 1.06873548e+01, 1.08254536e+01, 1.84808713e+01, 2.78611401e+01, 3.89863401e+01, 5.18564713e+01, 6.65135657e+01, 8.29422857e+01, 1.01074490e+02, 1.20437447e+02, 1.40424950e+02, 1.60705703e+02, 1.81127302e+02, 2.01663251e+02, 2.22260160e+02, 2.42872211e+02, 2.63526294e+02, 2.84165040e+02, 3.04803786e+02, 3.25465440e+02, 3.46123309e+02, 3.66731574e+02, 3.87301789e+02, 4.07818614e+02, 4.28293603e+02, 4.48692294e+02, 4.69049149e+02, 4.89352614e+02, 5.09614243e+02, 5.29799574e+02, 5.49943069e+02, 5.70038551e+02, 5.83886942e+02, 6.03453525e+02, 6.21652524e+02, 6.37750450e+02, 6.51852400e+02, 6.64485702e+02, 6.76047717e+02, 6.86812765e+02, 6.96938855e+02, 7.06603258e+02, 7.15873665e+02, 7.24866387e+02, 7.33614693e+02, 7.36947081e+02, 7.45470239e+02, 7.53909998e+02, 7.62234239e+02, 7.70463839e+02, 7.78605199e+02, 7.86685600e+02, 7.94732320e+02, 8.02745359e+02, 8.10703840e+02, 8.18628640e+02, 8.26526160e+02, 8.34417280e+02, 8.42281120e+02, 8.50117680e+02, 8.57947840e+02, 8.65750720e+02, 8.73526320e+02, 8.81298720e+02, 8.89067920e+02, 8.96809840e+02, 9.04524480e+02, 9.12235920e+02, 9.19944160e+02, 9.27637015e+02, 9.31738347e+02, 9.39451780e+02, 9.47132740e+02, 9.54786000e+02, 9.62433700e+02, 9.70056480e+02, 9.77673700e+02, 9.85266000e+02, 9.92852740e+02, 1.00041178e+03, 1.00794590e+03, 1.01549660e+03, 1.02300024e+03, 1.03049832e+03, 1.03799362e+03, 1.04546400e+03, 1.05293160e+03, 1.06039364e+03, 1.06780862e+03, 1.07524018e+03, 1.08264960e+03, 1.09005346e+03, 1.09742962e+03, 1.10478086e+03, 1.11215146e+03, 1.11949714e+03, 1.12681512e+03, 1.13413032e+03, 1.13866574e+03]
alpha = [-0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.85000002, -0.81570002, -0.12970001, 0.5563, 1.25, 1.95, 2.65, 3.35000001, 4.05000001, 4.75000001, 5.45000001, 6.15000001, 6.85000002, 7.55000002, 8.25000002, 8.95000002, 9.64987145, 10.34730002, 11.04472859, 11.53321429, 12.23178571, 12.93035712, 13.55799998, 14.16199998, 14.62449999, 14.6625, 14.7005, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.71, 14.58083199, 13.73334198, 13.187196, 12.999936, 12.92248399, 12.872484, 12.84124201, 12.82, 12.80124201, 12.79, 12.79242001, 12.84764402, 12.78869399, 12.76248401, 12.76000003, 12.76000003, 12.76000003, 12.76000003, 12.76124201, 12.76875799, 12.76999997, 12.77124199, 12.778758, 12.77999998, 12.75240002, 12.02190398, 11.078032, 10.62747203, 11.38190403, 11.96969599, 12.286688, 12.50852799, 12.68815999, 12.81889598, 12.91926401, 12.99926402, 13.05967918, 13.07881539, 13.11790001, 13.14105003, 13.16894999, 13.18999999, 13.20894999, 13.22000001, 13.23000001, 13.24, 13.24895, 13.25000001, 13.25105001, 13.25895, 13.26000001, 13.26000001, 13.26000001, 13.26000001, 13.26105001, 13.26895, 13.27000001, 13.27000001, 13.27000001, 13.27000001, 13.27000001, 13.27000001, 13.27000001, 13.26908801, 13.260912, 13.26091201, 13.26908801, 13.27000001, 13.27000001, 13.27000001, 13.27000001, 13.26544, 13.227296, 13.249088, 13.26817601, 13.27000001, 13.27000001, 13.26544, 13.227296, 13.249088, 13.26817601, 13.26544, 13.227296, 13.249088, 13.26908802, 13.27452801, 13.236384, 13.25000001, 13.263616, 13.227296, 13.23859998]
gamma = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.02475, 0.12375, 0.22275001, 0.46800001, 0.76200001, 1.06000001, 1.37000002, 1.68000002, 1.94500001, 2.19499999, 2.43649997, 2.65249998, 2.86849998, 3.07474998, 3.27774998, 3.47930998, 3.67654999, 3.69627399, 4.66751599, 5.62124199, 6.58, 7.54124199, 8.512484, 9.50248401,10.44044801,10.92522401,11.12130599,11.22379, 11.278758, 11.31627401,11.32875799,11.32999999,11.32751599,11.30999999,11.294968, 11.305032, 11.28751599,11.24875799,11.20124201,11.15999999,11.12000001,11.08000001,11.03999999,10.99999998,10.96000002,10.92, 10.87999998,10.841368, 10.8030347, 10.31086402, 9.30896001, 8.024352, 7.09993598, 6.44552, 5.94478401, 5.57257601, 5.27257598, 5.04184002, 4.86147199, 4.72110399, 4.61121723, 4.58282692, 4.52790002, 4.46104999, 4.40209998, 4.36000001, 4.32105001, 4.28999999, 4.26104999, 4.23999998, 4.22104997, 4.20895001, 4.19000001, 4.17105001, 4.15999999, 4.14999999, 4.14105001, 4.13789999, 4.12104999, 4.10999999, 4.09999999, 4.08999998, 4.08000002, 4.07000001, 4.06105001, 4.05801885, 4.05354476, 4.06635202, 4.05091201, 4.039088, 4.020912, 4.01, 4., 3.990912, 3.98908799, 3.98091199, 3.97817598, 3.95999997, 3.94182403, 3.93908803, 3.93000001, 3.92182401, 3.92726401, 3.91000001, 3.89182401, 3.89000001, 3.888176, 3.87091199, 3.86, 3.850912, 3.84908799, 3.83999999, 3.83182399, 3.83726399, 3.82759997]
t_s = [0., 0.32, 0.63, 0.95, 1.27, 1.59, 1.91, 2.23, 2.55, 2.87, 3.2, 3.52, 3.85, 4.18, 4.51, 4.84, 5.17, 5.5, 5.83, 6.17, 6.5, 6.84, 7.18, 7.52, 7.86, 8.2, 8.54, 8.89, 9.23, 9.58, 9.93, 10.28, 10.63, 10.98, 11.34, 11.69, 12.05, 12.41, 12.77, 13.13, 13.49, 13.86, 14.23, 14.59, 14.97, 15.34, 15.71, 16.09, 16.46, 16.84, 17.22, 17.61, 17.99, 18.38, 18.77, 19.16, 19.55, 19.95, 20.35, 20.75, 21.15, 21.55, 21.96, 22.37, 22.78, 23.19, 23.61, 24.03, 24.45, 24.87, 25.3, 25.73, 26.16, 26.59, 27.03, 27.47, 27.51, 27.71, 27.91, 28.11, 28.31, 28.51, 28.71, 28.91, 29.11, 29.31, 29.51, 29.71, 29.91, 30.11, 30.31, 30.51, 30.71, 30.91, 31.05, 31.25, 31.45, 31.65, 31.85, 32.05, 32.25, 32.45, 32.65, 32.85, 33.05, 33.25, 33.45, 33.65, 33.85, 34.05, 34.25, 34.27, 35.27, 36.27, 37.27, 38.27, 39.27, 40.27, 41.27, 42.27, 43.27, 44.27, 45.27, 46.27, 47.27, 48.27, 49.27, 50.27, 51.27, 52.27, 53.27, 54.27, 55.27, 56.27, 57.27, 58.27, 59.27, 60.27, 61.27, 62.27, 63.27, 64.27, 64.96, 65.96, 66.96, 67.96, 68.96, 69.96, 70.96, 71.96, 72.96, 73.96, 74.96, 75.96, 76.96, 77.35, 78.35, 79.35, 80.35, 81.35, 82.35, 83.35, 84.35, 85.35, 86.35, 87.35, 88.35, 89.35, 90.35, 91.35, 92.35, 93.35, 94.35, 95.35, 96.35, 97.35, 98.35, 99.35, 100.35, 101.35, 101.88, 102.88, 103.88, 104.88, 105.88, 106.88, 107.88, 108.88, 109.88, 110.88, 111.88, 112.88, 113.88, 114.88, 115.88, 116.88, 117.88, 118.88, 119.88, 120.88, 121.88, 122.88, 123.88, 124.88, 125.88, 126.88, 127.88, 128.88, 129.88]
M_0 = [0., 0.00354064, 0.00699206, 0.01058541, 0.0141, 0.01760506, 0.02112306, 0.02466119, 0.02819569, 0.03166812, 0.03523977, 0.03869764, 0.04226801, 0.0458, 0.04930097, 0.05283297, 0.056366, 0.0599, 0.063365, 0.066935, 0.0704, 0.073968, 0.0775, 0.08100165, 0.08452965, 0.08806471, 0.09159176, 0.09512265, 0.09856347, 0.102116, 0.105686, 0.1092, 0.11269856, 0.11619467, 0.11976131, 0.12323362, 0.1268, 0.1303, 0.1338345, 0.13737259, 0.1408967, 0.1444, 0.14793264, 0.15138158, 0.15499826, 0.1585, 0.16196968, 0.16553458, 0.16908463, 0.1726, 0.1760709, 0.17962031, 0.18310072, 0.18667661, 0.1902, 0.1937, 0.1971999, 0.20078888, 0.2043, 0.2078, 0.2113, 0.2148004, 0.21839115, 0.2219, 0.2254, 0.2289041, 0.23247957, 0.236, 0.2395, 0.24298136, 0.24652791, 0.25007112, 0.2536, 0.25708635, 0.26060518, 0.26418118, 0.2645049, 0.2661029, 0.2677009, 0.2693, 0.2709, 0.272498, 0.274058, 0.275618, 0.277178, 0.278738, 0.2802951, 0.2817971, 0.2832991, 0.2847571, 0.2861791, 0.28759369, 0.2888674, 0.29014112, 0.29098286, 0.29205714, 0.29313143, 0.2939475, 0.2946775, 0.295365, 0.295925, 0.296485, 0.2969775, 0.2974475, 0.29791, 0.29835, 0.29879, 0.2991775, 0.2995475, 0.2999001, 0.3002005, 0.30023054, 0.30146274, 0.30237516, 0.30306274, 0.30346274, 0.3035379, 0.30315032, 0.30247452, 0.3022621, 0.3025, 0.30281242, 0.30318758, 0.30351242, 0.3039, 0.3043, 0.30468758, 0.30501242, 0.30538758, 0.3057, 0.30601242, 0.3064, 0.3068, 0.30718758, 0.30751242, 0.3079, 0.3083, 0.30868758, 0.30901242, 0.3094, 0.3098, 0.31018632, 0.31040037, 0.3107, 0.311, 0.3113, 0.31159632, 0.3118, 0.312, 0.3122, 0.3124, 0.3126, 0.3128, 0.313, 0.31319264, 0.3132105, 0.3133105, 0.3134895, 0.3136105, 0.3137895, 0.3139105, 0.3140895, 0.3142105, 0.3143895, 0.3145105, 0.3147, 0.3148895, 0.3150105, 0.3151895, 0.3153, 0.3154105, 0.3155895, 0.3157105, 0.3158895, 0.3160105, 0.3161895, 0.3163105, 0.3164895, 0.3166105, 0.31679881, 0.31688279, 0.31691824, 0.31709088, 0.31720912, 0.31739088, 0.31750912, 0.31769088, 0.31780912, 0.31799088, 0.3181, 0.31820912, 0.3184, 0.31859088, 0.3187, 0.31880912, 0.31899088, 0.31910912, 0.31929088, 0.31940912, 0.31959088, 0.3197, 0.31980912, 0.31999088, 0.32010912, 0.32029088, 0.32040912, 0.32059088, 0.3207, 0.320762, ]
input_v = vcat(x[n_1:n_2], y[n_1:n_2], z[n_1:n_2], alpha[n_1:n_2], gamma[n_1:n_2], t_s[n_1:n_2], M_0[n_1:n_2])
if settings["core_jet_suppression"] && settings["case_name"] in ["nasa_stca_standard", "stca_enginedesign_standard"]
    TS = [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.89632, 0.8, 0.70041216, 0.61446784, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, 0.6112, ]
    input_v = vcat(input_v, TS[n_1:n_2])
end
if settings["atmosphere_type"] == "stratified"
    c_0 = [346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.16136, 346.158312, 346.14612, 346.133928, 346.13088, 346.13088, 346.130179, 346.1273748, 346.1270944, 346.1004, 346.0661344, 346.0127456, 345.9746944, 345.91752, 345.8527744, 345.7689056, 345.70416, 345.6394144, 345.55176, 345.4641056, 345.3955744, 345.3117056, 345.2431744, 345.1593056, 345.0907744, 345.00312, 344.9154656, 344.8469344, 344.7630656, 344.6945344, 344.6106656, 344.5421344, 344.4582656, 344.3897344, 344.30208, 344.2144256, 344.1458944, 344.0620256, 343.9938784, 343.9373279, 343.87536, 343.8144, 343.75344, 343.69248, 343.6326417, 343.5999183, 343.5412017, 343.5096, 343.47912, 343.44864, 343.41816, 343.3859257, 343.3622058, 343.32672, 343.29624, 343.26576, 343.23528, 343.2048, 343.17432, 343.14384, 343.11336, 343.08288, 343.0524, 343.02192, 342.99144, 342.96096, 342.93048, 342.9, 342.86952, 342.83904, 342.80856, 342.77808, 342.7476, 342.71712, 342.68664, 342.65616, 342.6288804, 342.6229002, 342.5952, 342.56472, 342.53424, 342.50376, 342.47328, 342.4428, 342.41232, 342.38184, 342.35136, 342.32088, 342.2904, 342.2626998, 342.2571402, 342.22944, 342.19896, 342.16848, 342.138, 342.10752, 342.07704, 342.04656, 342.01608, 341.9856, 341.95512, 341.92464, 341.8969398, 341.8913802, 341.86368, 341.8447824]
    T_0 = [298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2271955, 298.2219439, 298.2009374, 298.1799308, 298.1746792, 298.1746792, 298.1734714, 298.1686402, 298.168157, 298.1221676, 298.0631396, 297.9711804, 297.9056478, 297.8071948, 297.695724, 297.5513601, 297.4399365, 297.3285344, 297.1777486, 297.0270001, 296.9091666, 296.7649935, 296.6472121, 296.5031026, 296.3853731, 296.2348266, 296.0843175, 295.9666712, 295.8227272, 295.7051329, 295.5612524, 295.4437101, 295.2998933, 295.1824029, 295.0321623, 294.8819591, 294.764552, 294.6209006, 294.5042029, 294.4073809, 294.3013023, 294.196968, 294.092652, 293.9883546, 293.8859942, 293.8300247, 293.72961, 293.6755732, 293.6234592, 293.5713497, 293.5192449, 293.4641464, 293.4236052, 293.3629585, 293.3108722, 293.2587906, 293.2067135, 293.1546411, 293.1025733, 293.0505102, 292.9984516, 292.9463977, 292.8943484, 292.8423037, 292.7902637, 292.7382283, 292.6861975, 292.6341714, 292.5821498, 292.5301329, 292.4781206, 292.426113, 292.3741099, 292.3221115, 292.2701177, 292.2181285, 292.171602, 292.1614031, 292.114164, 292.0621887, 292.0102181, 291.958252, 291.9062906, 291.8543338, 291.8023816, 291.7504341, 291.6984911, 291.6465528, 291.5946191, 291.5474258, 291.5379543, 291.4907656, 291.4388459, 291.3869307, 291.3350202, 291.2831142, 291.2312129, 291.1793162, 291.1274241, 291.0755367, 291.0236539, 290.9717757, 290.9246328, 290.9151714, 290.8680332, 290.8358767]
    rho_0 = [1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1853717, 1.1842121, 1.18266596, 1.18137751, 1.18086213, 1.1808106, 1.18021791, 1.18021791, 1.18021791, 1.17957781, 1.17570422, 1.17506412, 1.17442402, 1.17055043, 1.16927023, 1.16539664, 1.16475654, 1.16411644, 1.16024285, 1.15896265, 1.15508906, 1.15380886, 1.14993527, 1.14929517, 1.14865507, 1.14478148, 1.14350128, 1.13962769, 1.13834749, 1.1344739, 1.1338338, 1.1331937, 1.12932011, 1.12803991, 1.12416632, 1.12352622, 1.12333656, 1.11856209, 1.11837243, 1.11818277, 1.1134083, 1.11321864, 1.11321864, 1.11302898, 1.10825451, 1.10806485, 1.10806485, 1.10806485, 1.10757837, 1.10429856, 1.10291106, 1.10291106, 1.10291106, 1.10291106, 1.10236991, 1.09829842, 1.09775727, 1.09775727, 1.09775727, 1.09775727, 1.09721612, 1.09314463, 1.09260348, 1.09260348, 1.09260348, 1.09260348, 1.09206233, 1.08799084, 1.08744969, 1.08744969, 1.08744969, 1.08744969, 1.08744969, 1.08642872, 1.08318276, 1.0822959, 1.0822959, 1.0822959, 1.0822959, 1.0822959, 1.08182587, 1.07761214, 1.07714211, 1.07714211, 1.07714211, 1.07714211, 1.07667208, 1.07245835, 1.07198832, 1.07198832, 1.07198832, 1.07198832, 1.07151829, 1.06730456, 1.06683453, 1.06683453, 1.06683453, 1.06683453, 1.06683453, 1.0663645, 1.06215077, 1.06168074, 1.06168074]
    p_0 = [101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101457.3923, 101455.6057, 101448.4593, 101441.3128, 101340.2919, 101207.9794, 101097.31, 101051.5694, 101046.9953, 100980.6979, 100960.7038, 100929.5551, 100852.6331, 100488.2267, 100395.9188, 100292.5758, 99924.34912, 99777.68027, 99396.70339, 99291.70844, 99197.77904, 98819.69222, 98671.47757, 98293.9163, 98145.9893, 97766.80905, 97662.73523, 97569.56358, 97193.23954, 97045.95517, 96670.15561, 96523.15823, 96147.88284, 96055.39447, 95952.31119, 95575.63494, 95429.27862, 95055.23708, 94963.47661, 94916.2312, 94478.76088, 94429.25026, 94379.75975, 93943.44684, 93894.73828, 93876.85634, 93828.78631, 93409.11018, 93376.55042, 93359.97887, 93343.4088, 93284.91385, 92995.84046, 92859.78318, 92843.29602, 92826.81034, 92810.32611, 92748.31489, 92389.34741, 92327.42014, 92311.01877, 92294.61885, 92278.22039, 92216.34346, 91857.82666, 91796.0336, 91779.71793, 91763.40372, 91747.09095, 91685.34817, 91327.28179, 91265.62284, 91249.39279, 91233.16419, 91216.93703, 91200.7113, 91100.58076, 90825.22551, 90736.18711, 90720.0426, 90703.89952, 90687.75788, 90671.61769, 90616.10937, 90247.09171, 90191.66662, 90175.609, 90159.5528, 90143.49804, 90089.58034, 89734.08423, 89680.23781, 89664.26413, 89648.29187, 89632.32103, 89577.05912, 89208.90153, 89153.72274, 89137.83436, 89121.94739, 89106.06182, 89090.17768, 89036.49905, 88681.78721, 88628.17977, 88618.3816, ]
    mu_0 = [1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.84e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.83e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.82e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.81e-05, 1.80e-05, 1.80e-05, 1.80e-05, 1.80e-05, 1.80e-05]
    I_0 = [409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.795638, 409.7953799, 409.7943475, 409.7933151, 409.7862821, 409.7772489, 409.7652153, 409.7441805, 409.7231457, 409.6942713, 409.6627837, 409.6290378, 409.588517, 409.5479962, 409.5013779, 409.4527271, 409.4016218, 409.343153, 409.3373061, 409.013188, 408.6160429, 408.1450203, 407.6001199, 406.9807073, 406.2885287, 405.5245789, 404.7091139, 403.8704715, 403.0201793, 402.1643519, 401.3073462, 400.4485758, 399.5895422, 398.7315711, 397.8747788, 397.0185818, 396.1650364, 395.3122415, 394.4620504, 393.6162036, 392.7729322, 391.9321665, 391.0980605, 390.2660853, 389.4369089, 388.6121645, 387.7908406, 386.9718787, 386.1579616, 385.5973371, 384.8052228, 384.0697156, 383.4196822, 382.8502468, 382.3401148, 381.8743725, 381.4418996, 381.0350966, 380.6468407, 380.2744136, 379.9131418, 379.5620955, 379.4287936, 379.0879236, 378.750389, 378.4174745, 378.0883448, 377.7627443, 377.4395815, 377.1183269, 376.7996778, 376.4832375, 376.1681357, 375.8541185, 375.5403559, 375.2276781, 374.9160849, 374.6055588, 374.2967126, 373.988946, 373.681306, 373.373792, 373.0673588, 372.7620049, 372.4567772, 372.1516769, 371.8471856, 371.6848502, 371.3795436, 371.0755228, 370.7725987, 370.4698941, 370.168176, 369.8666784, 369.5661672, 369.2658754, 368.9666807, 368.6684714, 368.3696069, 368.0726048, 367.7758221, 367.47915, 367.1834641, 366.8878889, 366.5925328, 366.2990402, 366.0048906, 365.7116177, 365.4185649, 365.1266082, 364.8356378, 364.5439019, 364.2531514, 363.9634983, 363.6739543, 363.494437, ]
    input_v = vcat(input_v, c_0[n_1:n_2], T_0[n_1:n_2], rho_0[n_1:n_2], p_0[n_1:n_2], mu_0[n_1:n_2], I_0[n_1:n_2])
end
if settings["fan_inlet_source"] == true || settings["fan_discharge_source"] == true
    DTt_f = [70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.38161814, 70.39354723, 70.59037718, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.67984534, 70.78273372, 70.91693595, 70.97359912, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97807253, 70.97682264, 70.97182309, 70.96682353, 70.96557364, 70.96557364, 70.96528618, 70.96413635, 70.96402136, 70.95307589, 70.93902723, 70.91714092, 70.90154417, 70.87811236, 70.85158231, 70.81722369, 70.79070488, 70.76419119, 70.72830415, 70.69242602, 70.66438166, 70.63006846, 70.60203647, 70.56773842, 70.5397188, 70.50388873, 70.46806758, 70.44006775, 70.40580907, 70.37782162, 70.34357808, 70.315603, 70.2813746, 70.2534119, 70.21765463, 70.18190627, 70.15396336, 70.11977433, 70.09200028, 69.85227716, 64.15772231, 58.27269063, 53.13151612, 52.91790382, 52.89947896, 52.88940443, 52.8713298, 52.86160317, 52.85222265, 52.84284296, 52.83346409, 52.82354635, 52.81624895, 52.80533253, 52.79595699, 52.78658229, 52.77720843, 52.7678354, 52.7584632, 52.74909183, 52.73972129, 52.73035159, 52.72098271, 52.71161468, 52.70224747, 52.6928811, 52.68351555, 52.67415085, 52.66478696, 52.65542392, 52.64606171, 52.63670033, 52.62733978, 52.61798007, 52.60862118, 52.59926313, 52.59088837, 52.58905255, 52.58054953, 52.57119397, 52.56183925, 52.55248536, 52.54313231, 52.53378008, 52.52442869, 52.51507813, 52.5057284, 52.49637951, 52.48703145, 52.47853664, 52.47683178, 52.46833782, 52.45899225, 52.44964752, 52.44030362, 52.43096055, 52.42161832, 52.41227692, 52.40293634, 52.39359661, 52.3842577, 52.37491963, 52.3664339, 52.36473086, 52.35624598, 52.3504578, ]
    mdot_f = [182.3948268, 182.3948268, 182.3948268, 182.3948268, 182.3948268, 182.4147701, 182.4856795, 182.6358697, 182.7887682, 182.7887682, 182.7887682, 182.7887682, 182.7887682, 182.7887682, 182.7925882, 182.9186493, 183.0487693, 183.1827095, 183.1827095, 183.1827095, 183.1827095, 183.1827095, 183.1827095, 183.1891979, 183.2995016, 183.4376127, 183.5766508, 183.5766508, 183.5766508, 183.5766508, 183.5766508, 183.7153181, 183.8623144, 183.9647393, 183.9705921, 184.0994984, 184.3152907, 184.3546849, 184.3645334, 184.3673016, 184.3749676, 184.6511523, 184.7584747, 184.7642241, 184.7884994, 185.036576, 185.152416, 185.2233254, 185.514842, 185.5463573, 185.5463573, 185.6330244, 185.9324198, 185.9402986, 186.0657539, 186.2243406, 186.33424, 186.33424, 186.4376495, 186.6050746, 186.8315909, 187.0778042, 187.1181832, 187.2145546, 187.3976893, 187.5160639, 187.5965405, 187.8863686, 187.9493994, 188.2137152, 188.3039465, 188.4177314, 188.5982115, 188.7639625, 189.066581, 189.0902175, 189.0997079, 189.2572844, 189.414861, 189.4857704, 189.4857704, 189.4936492, 189.6512257, 189.8088023, 189.8797117, 189.8797117, 189.8875905, 190.0451671, 190.2027436, 190.2736531, 190.2736531, 190.2787181, 190.3800172, 190.4813164, 190.5409703, 190.5972477, 190.6535251, 190.6675944, 190.6675944, 190.6675944, 190.6675944, 190.6675944, 190.6675944, 190.6675944, 190.7053062, 190.8561534, 191.0070006, 190.8578209, 190.6086321, 190.4005896, 190.3159857, 190.3075253, 190.1973331, 190.1785027, 190.1491633, 190.0251378, 189.3698208, 189.2312902, 189.082351, 188.4234134, 188.1820892, 187.511112, 187.3605742, 187.2204669, 186.5999277, 186.6457633, 186.0243192, 185.7812507, 185.1105139, 184.960462, 184.8207239, 184.1526596, 183.9101573, 183.2425701, 183.0003236, 182.3332136, 182.1940746, 182.044878, 181.4227307, 181.4611137, 180.8400717, 180.701297, 180.3680975, 172.4015026, 163.5900867, 156.3701253, 155.4043746, 155.3508488, 155.3360551, 155.2830501, 154.602723, 154.5625484, 154.5488327, 154.535117, 154.4527725, 153.984775, 153.7753932, 153.7617413, 153.7480894, 153.7344374, 153.6453628, 153.0642953, 152.9752878, 152.9616996, 152.9481115, 152.9345233, 152.8455527, 152.2648514, 152.2137215, 152.4843736, 152.5085892, 152.4950329, 152.4059562, 151.8242491, 151.7352396, 151.7217473, 151.7082549, 151.6947625, 151.6812701, 151.5267992, 151.0714403, 150.9355447, 150.9221162, 150.9086878, 150.8952594, 150.8818309, 150.8028832, 150.2021334, 150.1232526, 150.1098881, 150.0965236, 150.0831591, 150.0055284, 149.4160293, 149.3384568, 149.3251563, 149.3118557, 149.2985552, 149.2197995, 148.6197527, 148.5410639, 148.5278273, 148.546535, 148.8196503, 148.8383253, 148.7606982, 148.1704611, 148.0928922, 148.0847059]
    N_f = [8726.699976, 8727.205906, 8727.699131, 8728.212705, 8728.714928, 8729.215781, 8729.718507, 8730.224143, 8730.729264, 8731.225445, 8731.735876, 8732.230082, 8732.74026, 8733.244953, 8733.745216, 8734.250031, 8734.754928, 8735.259904, 8735.755086, 8736.265274, 8736.760456, 8737.270287, 8737.775041, 8738.275512, 8738.779638, 8739.284768, 8739.788767, 8740.293433, 8740.749672, 8741.179229, 8741.529953, 8741.866879, 8742.200216, 8742.533315, 8742.873037, 8743.203838, 8743.543653, 8743.877095, 8744.213793, 8744.550856, 8744.886644, 8745.220384, 8745.510877, 8745.712793, 8745.750809, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.75381, 8745.749886, 8745.718492, 8745.617365, 8745.506193, 8745.39502, 8745.283903, 8745.169819, 8745.058443, 8744.947266, 8744.835962, 8744.722431, 8744.610607, 8744.499543, 8744.388933, 8744.275388, 8744.129417, 8743.937441, 8743.716022, 8743.492428, 8743.265351, 8743.244794, 8743.143289, 8743.041784, 8742.940205, 8742.838565, 8742.737054, 8742.638003, 8742.538952, 8742.439903, 8742.340856, 8742.241988, 8742.146537, 8742.051086, 8741.958484, 8741.868215, 8741.778412, 8741.697486, 8741.61656, 8741.563093, 8741.494892, 8741.42669, 8741.374851, 8741.328465, 8741.284784, 8741.249219, 8741.213654, 8741.182387, 8741.152553, 8741.046212, 8740.710354, 8740.374496, 8740.272948, 8740.249503, 8740.251512, 8740.329878, 8740.337715, 8740.426361, 8740.169365, 8739.279359, 8738.5848, 8737.190798, 8735.297345, 8732.728852, 8730.95213, 8729.475363, 8727.469561, 8725.505397, 8723.990513, 8722.129997, 8720.66506, 8718.804329, 8717.289197, 8715.324347, 8713.317683, 8711.793987, 8709.93254, 8708.466827, 8706.605283, 8705.089478, 8703.227702, 8701.761579, 8699.803907, 8697.804476, 8696.329641, 8694.475475, 8693.009439, 8678.628624, 8335.069152, 7992.884158, 7697.406674, 7685.125837, 7683.977951, 7683.432902, 7682.306717, 7681.786666, 7681.291657, 7680.796615, 7680.30154, 7679.76033, 7679.246508, 7678.546208, 7678.031461, 7677.462656, 7676.947855, 7676.379005, 7675.86413, 7675.295088, 7674.780158, 7674.211219, 7673.706028, 7673.200807, 7672.631796, 7672.116746, 7671.5379, 7670.958907, 7670.443759, 7669.87464, 7669.359457, 7668.790294, 7668.275056, 7667.705848, 7667.190537, 7666.621156, 7666.186183, 7666.130503, 7665.543686, 7665.022354, 7664.450445, 7663.937556, 7663.365602, 7662.852641, 7662.280508, 7661.767626, 7661.186977, 7660.606426, 7660.101818, 7659.659391, 7659.636489, 7659.117937, 7658.604747, 7658.032563, 7657.519335, 7656.947107, 7656.433807, 7655.852918, 7655.272145, 7654.758783, 7654.186449, 7653.673033, 7653.162878, 7653.207233, 7652.688431, 7652.323017]
    A_f = [0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019, 0.87366019]
    d_f = [1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956, 1.10550956]
    input_v = vcat(input_v, DTt_f[n_1:n_2], mdot_f[n_1:n_2], N_f[n_1:n_2], A_f[n_1:n_2], d_f[n_1:n_2])
end
if settings["core_source"]
    if settings["core_turbine_attenuation_method"] == "ge"
        mdoti_c = [34.27289403, 34.27289403, 34.27289403, 34.27289403, 34.27289403, 34.34380347, 34.5959259, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.66683534, 34.76926008, 35.05289782, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06077665, 35.06570092, 35.10509504, 35.35130837, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.45471796, 35.56850287, 35.74898297, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.84865927, 35.93729607, 36.05547846, 36.15396379, 36.19335792, 36.23275205, 36.24260058, 36.24260058, 36.24228146, 36.24100497, 36.23972849, 36.20395778, 36.15668899, 36.11722525, 36.10117667, 36.09957181, 36.07866938, 36.07509742, 36.06953199, 36.04600553, 35.92169798, 35.89542001, 35.86716761, 35.74217327, 35.69639631, 35.56911815, 35.54056254, 35.51398546, 35.38721836, 35.34115695, 35.21448018, 35.16846719, 35.04149646, 35.01309157, 34.98663909, 34.86017424, 34.81426846, 34.68789392, 34.64203657, 34.51575235, 34.4894133, 34.46117032, 34.3346286, 34.28887845, 34.16280615, 34.13659, 34.05696789, 32.11420064, 29.90157201, 27.75853234, 27.55396714, 27.54447673, 27.54185373, 27.5324557, 27.41183032, 27.40470717, 27.40227531, 27.39984344, 27.38524336, 27.30226507, 27.26514064, 27.26272008, 27.26029953, 27.25787897, 27.2420856, 27.13905946, 27.12327798, 27.12086873, 27.11845949, 27.11605024, 27.1002753, 26.99731408, 26.98155104, 26.9791531, 26.97675517, 26.97435724, 26.95860074, 26.85570444, 26.83995983, 26.83757321, 26.83518659, 26.83279996, 26.83041334, 26.80308948, 26.72254251, 26.69850436, 26.69612905, 26.69375374, 26.69137843, 26.68900312, 26.6750383, 26.5687736, 26.55482062, 26.55245662, 26.55009262, 26.54772862, 26.56625741, 26.74993506, 26.76821396, 26.7658299, 26.76344584, 26.76106178, 26.74694519, 26.63938963, 26.62528504, 26.62291244, 26.62053984, 26.61816724, 26.61579465, 26.6019131, 26.49636479, 26.48249366, 26.48102976]
        Tti_c = [780.1623435, 780.3532089, 780.4605707, 780.4605707, 780.5763201, 780.6900193, 780.7437001, 780.7587979, 780.7886206, 780.979486, 781.057025, 781.0624473, 781.1519154, 781.2511437, 781.3552522, 781.3552522, 781.3552522, 781.3552522, 781.5520822, 781.6534794, 781.6534794, 781.7489121, 781.8506602, 781.9517066, 781.9517066, 782.0569633, 782.2176551, 782.242215, 782.2499338, 782.2976502, 782.5064092, 782.548161, 782.6257001, 782.8344591, 782.9390943, 783.0442407, 783.1520711, 783.205752, 783.3399542, 783.4449383, 783.4507416, 783.659823, 783.7410698, 783.7947507, 784.0214034, 784.1269918, 784.239351, 784.3912051, 784.6118932, 784.7211386, 784.8372902, 784.9356609, 784.9414725, 785.1624665, 785.3271798, 785.4472353, 785.5602557, 785.7988375, 785.9069448, 786.0336914, 786.205172, 786.3944736, 786.4489, 786.6533675, 786.8129554, 786.9358831, 787.0824925, 787.3019024, 787.349619, 787.5497151, 787.7012495, 787.8301114, 787.991709, 788.2133145, 788.4935912, 788.5114849, 788.5186694, 788.6379603, 788.7572512, 788.8732616, 788.986588, 789.0945462, 789.1005107, 789.1064753, 789.1747693, 789.2940602, 789.4100705, 789.4637515, 789.5174324, 789.5776743, 789.6432842, 789.7115782, 789.8308691, 789.95016, 790.0038409, 790.0038409, 790.0038409, 790.0038409, 790.0038409, 790.0336636, 790.1529545, 790.2722454, 790.3020681, 790.3020681, 790.2881513, 790.232484, 790.1768166, 790.1628998, 790.1628998, 790.1665568, 790.181185, 790.1826478, 790.2848378, 790.1653831, 789.9215991, 789.7478723, 789.4868734, 789.1913643, 788.8086555, 788.5132716, 788.2179447, 787.8182113, 787.4185772, 787.1430731, 786.9839002, 786.7084064, 786.3262281, 786.0140095, 785.6147602, 785.2156102, 784.9036121, 784.5586099, 784.4689866, 784.1240027, 783.8121629, 783.4306169, 783.118915, 782.7203266, 782.3584581, 782.2685068, 781.9238701, 781.6141543, 780.5121388, 758.1744553, 736.4509349, 717.4290242, 716.7327879, 716.4940539, 716.3576, 716.1127892, 715.9810475, 715.8539934, 715.7269507, 715.5999192, 715.4655889, 715.3667496, 715.2188928, 715.0919064, 714.9649313, 714.8379675, 714.7417954, 714.8463986, 714.7501942, 714.6232235, 714.496264, 714.3693158, 714.2423789, 714.1154532, 713.9885389, 713.8616358, 713.7347439, 713.6078634, 713.4809941, 713.3541361, 713.2272894, 713.100454, 713.0043229, 713.1083969, 713.0122336, 712.8987091, 712.8738236, 712.7585602, 712.6317405, 712.504932, 712.3781349, 712.251349, 712.1245744, 711.9978111, 711.8710591, 711.7443184, 711.6175889, 711.4908707, 711.375719, 711.3526085, 711.2640516, 711.3756417, 711.2754978, 711.1487841, 711.0220818, 710.8953906, 710.7687109, 710.6420423, 710.5153851, 710.3887392, 710.2621045, 710.1470286, 710.1239334, 710.0088691, 709.930375, ]
        Ttj_c = [1687.965927, 1688.076752, 1688.183723, 1688.293586, 1688.446632, 1688.63116, 1688.875706, 1689.043086, 1689.188658, 1689.379524, 1689.568898, 1689.742819, 1689.844588, 1689.949408, 1690.062373, 1690.354636, 1690.548574, 1690.649971, 1690.846801, 1691.049596, 1691.246426, 1691.341859, 1691.443607, 1691.561494, 1691.847792, 1692.035851, 1692.164965, 1692.373725, 1692.545163, 1692.689846, 1692.779313, 1693.035789, 1693.256354, 1693.34549, 1693.634819, 1693.830096, 1693.960294, 1694.175017, 1694.331586, 1694.524771, 1694.817034, 1694.906399, 1695.12616, 1695.364823, 1695.439863, 1695.719834, 1696.015477, 1696.26402, 1696.339518, 1696.614515, 1696.816054, 1696.980035, 1697.2125, 1697.439458, 1697.604171, 1697.807424, 1698.101063, 1698.370232, 1698.482163, 1698.702106, 1699.000333, 1699.267919, 1699.322345, 1699.596787, 1699.895015, 1700.107556, 1700.254166, 1700.491469, 1700.789696, 1701.019615, 1701.171149, 1701.38615, 1701.684377, 1701.981441, 1702.261718, 1702.279612, 1702.289481, 1702.462452, 1702.635424, 1702.749346, 1702.814956, 1702.883251, 1703.002542, 1703.121833, 1703.241124, 1703.360415, 1703.479706, 1703.598996, 1703.718287, 1703.834298, 1703.947624, 1704.059417, 1704.142068, 1704.22472, 1704.272564, 1704.315168, 1704.357771, 1704.435523, 1704.524992, 1704.599549, 1704.629372, 1704.659194, 1704.66665, 1704.66665, 1704.636632, 1704.516558, 1704.396484, 1704.455919, 1704.575189, 1704.664594, 1704.664405, 1704.664386, 1704.662551, 1704.586073, 1704.09718, 1703.7224, 1703.159347, 1702.521846, 1701.659276, 1700.835441, 1700.384955, 1699.559544, 1698.697414, 1698.060396, 1697.4589, 1696.822053, 1696.034569, 1695.583904, 1694.759443, 1693.898381, 1693.262082, 1692.660899, 1692.02477, 1691.238191, 1690.787654, 1690.001289, 1689.36555, 1688.727449, 1687.904334, 1687.232295, 1686.446623, 1685.99998, 1682.882101, 1614.481368, 1549.79568, 1494.031953, 1491.413741, 1491.166718, 1490.893545, 1490.384041, 1490.109859, 1489.845432, 1489.591832, 1489.599366, 1489.330543, 1489.124797, 1488.817014, 1488.552676, 1488.288362, 1488.054857, 1488.022175, 1487.788662, 1487.52439, 1487.26014, 1486.995914, 1486.731713, 1486.498282, 1486.465424, 1486.231985, 1485.967825, 1485.703688, 1485.439575, 1485.175485, 1484.942127, 1484.909095, 1484.67573, 1484.411682, 1484.147657, 1483.883657, 1483.647395, 1483.595605, 1483.382365, 1483.357219, 1483.119897, 1482.855962, 1482.592049, 1482.328162, 1482.064296, 1481.827062, 1481.80173, 1481.564489, 1481.300666, 1481.060923, 1481.012808, 1480.77309, 1480.535916, 1480.510418, 1480.273237, 1480.009503, 1479.745792, 1479.482105, 1479.218442, 1478.954802, 1478.717726, 1478.692026, 1478.478984, 1478.430901, 1478.191345, 1478.027925]
        Pti_c = [2205379.337, 2206097.786, 2206792.853, 2207509.011, 2208180.766, 2208852.287, 2209544.226, 2210249.291, 2210958.619, 2211663.226, 2212387.968, 2213088.807, 2213799.897, 2214510.502, 2215219.659, 2215895.365, 2216572.116, 2217249.852, 2217952.951, 2218677.357, 2219380.457, 2220090.659, 2220800.86, 2221511.063, 2222221.263, 2222931.465, 2223631.88, 2224277.328, 2225061.21, 2226030.38, 2227286.423, 2228511.622, 2229728.609, 2230944.745, 2232206.14, 2233437.504, 2234700.523, 2235918.012, 2237135.501, 2238379.131, 2239669.01, 2240889.424, 2242153.267, 2243471.364, 2245055.717, 2246672.496, 2248249.231, 2249845.068, 2251439.497, 2253035.262, 2254605.571, 2256218.647, 2257831.976, 2259432.401, 2261010.298, 2262601.306, 2264223.747, 2265881.664, 2267775.633, 2269703.324, 2271657.646, 2273643.458, 2275586.368, 2277515.542, 2279443.233, 2281386.192, 2283379.344, 2285327.762, 2287255.452, 2289172.875, 2291139.147, 2293198.59, 2295397.714, 2297679.445, 2299949.801, 2302360.429, 2302576.432, 2303609.268, 2304642.104, 2305677.172, 2306714.067, 2307750.759, 2308783.595, 2309816.431, 2310850.383, 2311885.249, 2312917.172, 2313893.192, 2314869.212, 2315822.911, 2316758.349, 2317688.945, 2318527.563, 2319366.181, 2319916.399, 2320607.759, 2321299.118, 2321829.595, 2322306.445, 2322755.394, 2323120.64, 2323485.887, 2323805.478, 2324109.85, 2324370.742, 2324501.195, 2324631.648, 2322564.546, 2319764.925, 2317478.733, 2316732.823, 2316658.232, 2316459.571, 2317010.519, 2317014.715, 2315728.012, 2307436.295, 2304852.115, 2301789.434, 2293126.012, 2289997.494, 2281564.213, 2279539.031, 2277779.341, 2269492.981, 2266496.027, 2258278.24, 2255210.68, 2246864.6, 2244790.089, 2243026.637, 2234764.087, 2231777.817, 2223582.836, 2220527.191, 2212266.567, 2210534.711, 2208606.401, 2200258.502, 2197270.925, 2189110.226, 2187375.721, 2180084.864, 2003269.891, 1825660.162, 1671999.78, 1658755.961, 1658087.182, 1657959.159, 1657297.85, 1650071.916, 1649683.516, 1649574.03, 1649381.468, 1648524.128, 1643432.245, 1641120.697, 1640995.507, 1640816.445, 1640691.198, 1639707.188, 1633526.322, 1632543.136, 1632418.361, 1632240.024, 1632124.882, 1631205.204, 1624974.577, 1624045.787, 1623858.549, 1623671.3, 1623546.887, 1622565.223, 1616391.942, 1615411.106, 1615287.169, 1615110.284, 1614986.29, 1614809.37, 1613207.975, 1608408.249, 1606855.412, 1606651.953, 1606465.036, 1606343.984, 1606165.304, 1605346.72, 1598915.935, 1598097.914, 1597911.79, 1597725.657, 1597613.214, 1596829.587, 1590626.362, 1589769.754, 1589649.56, 1589472.389, 1589352.141, 1588478.152, 1582111.705, 1581230.426, 1581045.896, 1580926.096, 1580749.662, 1580629.806, 1579782.716, 1573650.199, 1572795.677, 1572676.743]
        DTt_des_c = [807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3904864, 807.3762687, 807.3193976, 807.2625266, 807.2483089, 807.2483089, 807.245039, 807.2319594, 807.2306514, 807.1061443, 806.9463378, 806.6973766, 806.5199602, 806.2534185, 805.9516335, 805.560797, 805.25914, 804.9575412, 804.5493186, 804.1411974, 803.8221869, 803.4318671, 803.1129972, 802.7228497, 802.4041206, 801.9965461, 801.5890729, 801.270569, 800.8808693, 800.5625062, 800.1729787, 799.8547563, 799.4654011, 799.1473195, 798.740573, 798.333928, 798.0160715, 797.6271641, 797.3112283, 797.0491023, 796.7619158, 796.4794513, 796.1970368, 795.9146724, 795.6375521, 795.4860257, 795.2141731, 795.0678793, 794.926791, 794.7857152, 794.644652, 794.4954836, 794.3857265, 794.2215375, 794.0805243, 793.9395236, 793.7985355, 793.6575598, 793.5165967, 793.3756461, 793.2347081, 793.0937825, 792.9528695, 792.8119689, 792.671081, 792.5302055, 792.3893425, 792.2484921, 792.1076542, 791.9668288, 791.8260159, 791.6852155, 791.5444277, 791.4036524, 791.2628896, 791.1221393, 790.9961782, 790.9685665, 790.8406763, 790.6999636, 790.5592634, 790.4185757, 790.2779005, 790.1372379, 789.9965877, 789.8559502, 789.715325, 789.5747125, 789.4341124, 789.3063458, 789.2807037, 789.1529498, 789.0123874, 788.8718374, 788.7313, 788.590775, 788.4502626, 788.3097627, 788.1692754, 788.0288005, 787.8883382, 787.7478884, 787.6202583, 787.5946436, 787.4670263, 787.379969, ]
        input_v = vcat(input_v, mdoti_c[n_1:n_2], Tti_c[n_1:n_2], Ttj_c[n_1:n_2], Pti_c[n_1:n_2], DTt_des_c[n_1:n_2])
    end
end

if settings["jet_mixing_source"] == true && settings["jet_shock_source"] == false
    V_j = [410.5127568, 410.5606387, 410.6159338, 410.6857467, 410.7550698, 410.8260545, 410.9015177, 410.9839474, 411.0713551, 411.1709237, 411.2625109, 411.3454323, 411.4479826, 411.5633252, 411.6886775, 411.7932183, 411.9091719, 412.0358668, 412.1615234, 412.2909878, 412.4166443, 412.5551088, 412.6935734, 412.8334227, 412.9954262, 413.1558006, 413.3134871, 413.4638639, 413.6264026, 413.8012896, 413.9743704, 414.159636, 414.345562, 414.5274507, 414.7135646, 414.9070644, 415.116703, 415.3243997, 415.5320965, 415.7397527, 415.9473373, 416.1646175, 416.3878168, 416.6097718, 416.8483237, 417.0796435, 417.3068156, 417.5472113, 417.8134821, 418.0590745, 418.3103082, 418.5722631, 418.8211799, 419.0975586, 419.3744877, 419.6514167, 419.927547, 420.2051861, 420.482204, 420.7699506, 421.0615916, 421.3506469, 421.6594228, 421.9706979, 422.2822431, 422.5916571, 422.9124051, 423.2494178, 423.5665018, 423.8996132, 424.2304069, 424.566908, 424.9130695, 425.2578806, 425.6055336, 425.9537721, 425.9861696, 426.1592503, 426.332331, 426.4977961, 426.6570303, 426.8162645, 426.9754988, 427.134733, 427.3015828, 427.4746635, 427.6470518, 427.8062861, 427.9655203, 428.1175197, 428.2635998, 428.4096354, 428.5548253, 428.7000152, 428.7946508, 428.9118512, 429.0290516, 429.1180645, 429.1976817, 429.2738372, 429.3396078, 429.4053784, 429.4581681, 429.5066307, 429.5582326, 429.6192527, 429.6802727, 429.7214875, 429.7561006, 429.7929581, 429.8365491, 429.8409082, 429.995136, 430.125597, 430.2236518, 430.2887266, 430.3170977, 430.310021, 430.2748252, 430.2719837, 430.3296543, 430.4192688, 430.5044942, 430.5484509, 430.6296481, 430.6994336, 430.7847532, 430.8501158, 430.9131787, 430.9935121, 431.0370302, 431.1176472, 431.1826721, 431.2415047, 431.302121, 431.3608057, 431.4212955, 431.4793638, 431.5373481, 431.6019124, 431.6818016, 431.7242657, 431.0109115, 410.9267218, 387.3808223, 364.7296203, 363.9015975, 363.9400659, 363.9766579, 364.0137285, 364.0183872, 364.0535192, 364.0899032, 364.126275, 364.1582468, 364.1428297, 364.1666515, 364.1993762, 364.2085754, 364.2376749, 364.2468577, 364.275937, 364.2851034, 364.3141627, 364.3233124, 364.3559538, 364.3885836, 364.3977071, 364.4267147, 364.4322209, 364.4377202, 364.4666994, 364.475783, 364.5047421, 364.5138092, 364.5427482, 364.5517989, 364.5807178, 364.589752, 364.625247, 364.6476569, 364.6303202, 364.65702, 364.6650757, 364.6948797, 364.7029191, 364.7327029, 364.7407259, 364.7704896, 364.7753739, 364.7802515, 364.813108, 364.8457949, 364.8492329, 364.8539269, 364.88363, 364.8915875, 364.9212702, 364.9292114, 364.958874, 364.9636794, 364.9653588, 364.9701511, 364.9997762, 365.0045531, 365.0091728, 365.0374266, 365.0451552, 365.0642474]
    rho_j = [0.81104317, 0.81101919, 0.81099647, 0.81097375, 0.81095559, 0.81093649, 0.81091278, 0.81089367, 0.81087558, 0.81085325, 0.81083398, 0.8108175, 0.81079781, 0.81078239, 0.8107703, 0.81075062, 0.81073497, 0.81072312, 0.81070356, 0.81068744, 0.81067571, 0.81066385, 0.810652, 0.81064014, 0.81062829, 0.81061644, 0.81060492, 0.81059564, 0.81058852, 0.81058126, 0.81057106, 0.81056485, 0.81055734, 0.81054596, 0.81054163, 0.81053745, 0.81053317, 0.81053103, 0.8105257, 0.81052161, 0.81052161, 0.81051838, 0.81051373, 0.81050975, 0.81050975, 0.81050975, 0.81050975, 0.81050998, 0.8105109, 0.81051821, 0.81052161, 0.81052167, 0.8105219, 0.81053069, 0.81053346, 0.81053677, 0.8105441, 0.8105465, 0.81055406, 0.81056087, 0.81056902, 0.81058073, 0.81059166, 0.81059551, 0.81060458, 0.81061636, 0.81062829, 0.81064015, 0.810652, 0.81066379, 0.8106757, 0.81068756, 0.81069941, 0.81071321, 0.81073416, 0.81074602, 0.81074717, 0.81075405, 0.81076092, 0.81076806, 0.81077541, 0.81078286, 0.81079235, 0.81080183, 0.81081118, 0.81082043, 0.81082957, 0.81083668, 0.81084379, 0.8108509, 0.81085802, 0.81086502, 0.81087, 0.81087498, 0.81087989, 0.81089055, 0.81090122, 0.81091811, 0.81093708, 0.81096079, 0.81099872, 0.81103665, 0.8110897, 0.81114778, 0.81114185, 0.8109439, 0.81074594, 0.80996806, 0.80899688, 0.8081909, 0.80788048, 0.80784944, 0.80742534, 0.80735652, 0.80700139, 0.80666946, 0.80383331, 0.80317145, 0.80221617, 0.79957223, 0.79874364, 0.79567608, 0.79483079, 0.79440524, 0.79145044, 0.79060864, 0.78766772, 0.78682751, 0.7838103, 0.78301286, 0.78261511, 0.77969731, 0.77886354, 0.77594921, 0.77512268, 0.77220186, 0.77180118, 0.77100239, 0.76800635, 0.76717766, 0.76425554, 0.76387002, 0.76421753, 0.77787244, 0.79502745, 0.8096006, 0.80648209, 0.80603613, 0.8062331, 0.8056663, 0.80231962, 0.80228205, 0.80234935, 0.80240394, 0.80203764, 0.79941891, 0.79831896, 0.79832011, 0.79831896, 0.79830908, 0.79790521, 0.79493742, 0.79452378, 0.79450067, 0.79446889, 0.79443711, 0.79402124, 0.79104319, 0.7906188, 0.79058603, 0.7905521, 0.79051069, 0.79008526, 0.78710584, 0.78667198, 0.78662962, 0.78659472, 0.78655236, 0.78651, 0.78580366, 0.78364625, 0.78302043, 0.78298598, 0.78294368, 0.78290039, 0.78285808, 0.78248367, 0.77939278, 0.77900093, 0.77895686, 0.77891377, 0.77886971, 0.77853491, 0.77595261, 0.77562545, 0.77557283, 0.77552021, 0.77547537, 0.77508374, 0.77199109, 0.77159874, 0.7715454, 0.77149206, 0.77143872, 0.77138538, 0.7710486, 0.76845231, 0.7681154, 0.76808249]
    A_j = [0.5822476, 0.58224403, 0.58223809, 0.58222841, 0.5822188, 0.58220871, 0.58219738, 0.58218412, 0.58216985, 0.58215418, 0.58213828, 0.58212283, 0.5821036, 0.58208439, 0.58206499, 0.58203946, 0.58201392, 0.58198838, 0.5819662, 0.58194009, 0.58191158, 0.58188277, 0.58185397, 0.58182479, 0.58178946, 0.58175458, 0.58171966, 0.58168091, 0.58164316, 0.58160436, 0.58156595, 0.58152755, 0.58148667, 0.58144159, 0.58139943, 0.58135519, 0.58130674, 0.58125874, 0.58121073, 0.58116024, 0.58110535, 0.58105451, 0.58100229, 0.58095015, 0.5808937, 0.58083631, 0.58077921, 0.5807191, 0.58065479, 0.58059355, 0.58053268, 0.580469, 0.58040426, 0.58033708, 0.58026987, 0.58020267, 0.5801347, 0.58005964, 0.57998894, 0.57991465, 0.57984037, 0.57976881, 0.57968493, 0.57960743, 0.57953063, 0.57945165, 0.57936938, 0.57928997, 0.57920414, 0.57911826, 0.5790314, 0.57894499, 0.57885858, 0.57877091, 0.57867657, 0.57858632, 0.57857801, 0.5785348, 0.5784916, 0.57844787, 0.57840371, 0.57835974, 0.57831942, 0.57827909, 0.57823666, 0.5781925, 0.57814842, 0.57810598, 0.57806355, 0.57802333, 0.57798493, 0.57794663, 0.5779102, 0.57787377, 0.57784949, 0.57781795, 0.5777864, 0.57776124, 0.5777382, 0.57771708, 0.57770171, 0.57768635, 0.57767315, 0.57766067, 0.57765083, 0.57764891, 0.57764699, 0.57763931, 0.57762971, 0.57762131, 0.57761651, 0.57761603, 0.57761051, 0.57761647, 0.57765613, 0.57769089, 0.5777473, 0.57782768, 0.57793938, 0.57801254, 0.57807491, 0.57815774, 0.57824057, 0.57830414, 0.57837498, 0.57843133, 0.57850819, 0.57856454, 0.57863777, 0.57871219, 0.57877457, 0.57884422, 0.57889216, 0.57896061, 0.57901577, 0.57908423, 0.57913098, 0.5791958, 0.57926664, 0.57931339, 0.57937463, 0.57942597, 0.57944124, 0.57891972, 0.57847061, 0.57817971, 0.57820574, 0.57824344, 0.57826405, 0.57830104, 0.57832095, 0.57834015, 0.578359, 0.57836931, 0.57838926, 0.5784042, 0.57842656, 0.57844576, 0.57846496, 0.57848416, 0.57850336, 0.57852256, 0.57854076, 0.57855238, 0.57857057, 0.57858876, 0.57860038, 0.57861857, 0.57863777, 0.57865697, 0.57867517, 0.57868678, 0.57870498, 0.57872317, 0.57873479, 0.57875298, 0.57877117, 0.57878279, 0.57880098, 0.57881627, 0.57881399, 0.57882978, 0.57884811, 0.57885946, 0.57887779, 0.57889611, 0.57890746, 0.57892491, 0.57893627, 0.57895372, 0.57896507, 0.57898252, 0.57899212, 0.57899387, 0.57900347, 0.57902092, 0.57903227, 0.57904972, 0.5790602, 0.57907067, 0.57908812, 0.57909948, 0.57911693, 0.5791274, 0.57913788, 0.57915358, 0.57914923, 0.57916493, 0.57917176]
    Tt_j = [517.4838297, 517.5441677, 517.5947882, 517.6358474, 517.6925887, 517.7522342, 517.8118795, 517.871525, 517.931097, 517.9902725, 518.0511391, 518.1099983, 518.1697183, 518.2293975, 518.2896394, 518.368968, 518.4381567, 518.4978021, 518.5666926, 518.6376707, 518.7065612, 518.7864862, 518.8657795, 518.94395, 519.0131387, 519.0942565, 519.1811283, 519.25051, 519.3319046, 519.4223065, 519.5117746, 519.5907451, 519.6690482, 519.7506632, 519.8405971, 519.9284562, 520.0195063, 520.1143426, 520.2172311, 520.3169706, 520.4063903, 520.5040489, 520.6060386, 520.7041876, 520.7965332, 520.9046729, 521.0021355, 521.0987356, 521.2119835, 521.3224218, 521.4291057, 521.5338313, 521.6297228, 521.7484786, 521.8677694, 521.9870602, 522.1030248, 522.1987634, 522.3151102, 522.4344011, 522.553692, 522.6726555, 522.7922446, 522.9185621, 523.0517169, 523.179153, 523.29926, 523.418764, 523.5410371, 523.679489, 523.8145687, 523.9467459, 524.0746507, 524.1984783, 524.3424451, 524.4826119, 524.4950913, 524.5535439, 524.6119964, 524.677338, 524.7483161, 524.8187574, 524.8789993, 524.9392412, 524.999155, 525.0588004, 525.1184459, 525.1780913, 525.2377368, 525.2973822, 525.3570276, 525.4165197, 525.4730977, 525.5296757, 525.5658933, 525.6089232, 525.651953, 525.6850775, 525.7149003, 525.7417407, 525.7596343, 525.777528, 525.7976583, 525.8185342, 525.8338767, 525.8326189, 525.8313611, 525.839992, 525.851919, 525.8637731, 525.8754085, 525.876572, 525.8838001, 525.8503934, 525.7888411, 525.7214957, 525.6259984, 525.4924904, 525.2971694, 525.1636444, 525.056149, 524.9087438, 524.7762336, 524.6495575, 524.5247511, 524.416623, 524.2731005, 524.1612107, 524.0097809, 523.8583028, 523.7462945, 523.6025501, 523.4904666, 523.3466359, 523.2381463, 523.1164223, 523.0078449, 522.8559941, 522.7040954, 522.5917323, 522.4475808, 522.3375813, 521.5500836, 503.212735, 484.691224, 469.0501135, 468.3528473, 468.2474728, 468.1898424, 468.0864253, 468.0307609, 467.9770691, 467.9244546, 467.8979481, 467.8422311, 467.7973529, 467.7147251, 467.6811055, 467.6304671, 467.5767458, 467.5230213, 467.4692939, 467.4155634, 467.3618299, 467.3080934, 467.2543537, 467.2006112, 467.1468655, 467.0931168, 467.0393651, 466.9856104, 466.9318526, 466.8780918, 466.8243281, 466.7705612, 466.7167914, 466.6630185, 466.6092426, 466.5554637, 466.5073291, 466.4967768, 466.4478969, 466.394109, 466.340318, 466.286524, 466.2327271, 466.1789271, 466.1277853, 466.0978322, 466.0440188, 465.9690169, 465.9363821, 465.8901274, 465.8803098, 465.8313924, 465.7775632, 465.7237311, 465.6698959, 465.6160578, 465.5622167, 465.5083726, 465.4545255, 465.4006754, 465.3468223, 465.2929663, 465.2466726, 465.2579805, 465.1878982, 465.1518494]
    input_v = vcat(input_v, V_j[n_1:n_2], rho_j[n_1:n_2], A_j[n_1:n_2], Tt_j[n_1:n_2])
end
if settings["airframe_source"]
    theta_flaps = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
    I_landing_gear = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    
    input_v = vcat(input_v, theta_flaps[n_1:n_2], I_landing_gear[n_1:n_2])
end


In [23]:
include("../src/noise_src_jl/get_noise.jl")
get_noise_fwd! = (y,x) -> get_noise!(y, x, settings, pyna_ip, af, data, sealevel_atmosphere, n_t, n_t_target)

#189 (generic function with 1 method)

In [24]:
# Compute
n_t_target = 10
output_v = zeros(2, 1)

@btime get_noise_fwd!(output_v, input_v)

println(output_v)

[42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278, 42.81479759298278][1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093][0.08265973954989855, 0.08265973954989855, 0.08265973954989855, 

[41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245, 41.97124361561245][1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661][0.08284160725317402, 0.08284160725317402, 0.08284160725317402, 0.08284160725317402, 0.0

[41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473][1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252][0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.0

[41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718, 41.91752803703718][1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093][0.08265973954989855, 0.08265973954989855, 0.08265973954989855, 

[41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409, 41.02455738288409][1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661, 1.753434963674661][0.08284160725317402, 0.08284160725317402, 0.08284160725317402, 0.08284160725317402, 0.0

...

[41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473, 41.55530174465473][1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252][0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.0

[40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207, 40.97402961695207][1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866, 1.7535149639798866][0.08293251255278816, 0.08293251255278816, 0.08293251255278816, 

[41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506, 41.17320010640506][1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252, 1.753408296906252][0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.08281130125769048, 0.0

Excessive output truncated after 524305 bytes.

0.7938304063975878, 0.8938304063975874, 0.9938304063975879, 1.0938304063975877, 1.1938304063975875, 1.2938304063975878, 1.3938304063975875]
[41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073, 41.6503538136073][1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.7532749630642093, 1.75327

In [6]:
# Compute partials
println("\n--- Compute partials ----")
Y = zeros(2, 1)
X = input_v
J = Y.*X'
#'

b_tape   = JacobianTape(get_noise_fwd!, Y, X)
b_tape_c = compile(b_tape)


--- Compute partials ----


ReverseDiff.CompiledTape(#125)

In [7]:
@time jacobian!(J, b_tape_c, input_v)
println(J)

 31.930672 seconds (181.02 M allocations: 6.865 GiB, 5.48% gc time)
[1.7180789256173765 -1.7073298180691454 0.0 0.0 0.0 0.28991860018315374 -0.2881040662147854 0.0 0.0 0.0 -28.92881828586433 28.748379832901236 0.0 0.0 0.0 -4.918475036114688 4.887796517574081 0.0 0.0 0.0 -4.918475036114688 4.887796517574081 0.0 0.0 0.0 -103.09640028785789 102.45719324465392 0.0 0.0 -0.007252549282671156 1443.9881442357337 -1434.980978421669 0.0 0.0 0.0 -4.956185930590636 4.925394325644275 0.0 0.0 0.0 -1.1718508454961258 1.1646099141991733 0.0 0.0 0.0 -1006.628825309304 1000.3534045943809 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.9823832363670637 1.9700175348586026 0.0 0.0 0.0 9.75857399396479 -9.697707337092123 0.0 0.0 0.0 1291.625017561289 -1283.5687385392662 0.0 0.0 0.0 2143.61947047784 -2130.248547841516 0.0 0.0 0.0 0.6287997358007421 -0.6248774262044104 0.0 0.0 0.0; 1.2148970776955321 -1.2073263583852034 0.0 0.0 0.0 0.06889118224902482 -0.06846187764733727 0.0 0.0 0.0 -47.6879299594593 

In [12]:
data.f_sb

24-element Array{Float64,1}:
    50.11
    63.09
    79.43
   100.0
   125.89
   158.49
   199.53
   251.19
   316.23
   398.11
   501.19
   630.96
   794.33
  1000.0
  1258.93
  1584.89
  1995.26
  2511.89
  3162.28
  3981.07
  5011.87
  6309.57
  7943.28
 10000.0

get_subband_frequencies! (generic function with 1 method)

In [12]:
f_sb = [   45.69436719,    47.8554422 ,    50.11872336,    52.48904442,   54.97146773,    57.52580003,    60.24643228,    63.09573445,   66.07979186,    69.20497765,    72.42069149,    75.84576457,   79.43282347,    83.18952918,    87.12390499,    91.17224886,   95.48416039,   100.        ,   104.72941228,   109.68249797,  114.77906094,   120.20743594,   125.89254118,   131.84651848,  138.08208392,   144.49827655,   151.33219579,   158.48931925,  165.98493258,   173.83504436,   181.91255231,   190.5159469 ,  199.5262315 ,   208.9626496 ,   218.84535481,   229.01433483,  239.84536691,   251.18864315,   263.06838969,   275.50997842,  288.31196578,   301.9474273 ,   316.22776602,   331.18348082,  346.84651304,   362.96326025,   380.12928926,   398.10717055,  416.93529997,   436.65388926,   456.94367188,   478.55442202,  501.18723363,   524.89044421,   549.71467735,   575.25800028,  602.4643228 ,   630.95734448,   660.79791862,   692.04977655,  724.20691489,   758.45764568,   794.32823472,   831.89529182,  871.23904992,   911.72248856,   954.84160391,  1000.        , 1047.29412282,  1096.82497969,  1147.79060935,  1202.0743594 , 1258.92541179,  1318.46518484,  1380.82083923,  1444.98276553, 1513.32195792,  1584.89319246,  1659.84932576,  1738.35044364, 1819.12552313,  1905.15946905,  1995.26231497,  2089.62649595, 2188.4535481 ,  2290.14334831,  2398.4536691 ,  2511.88643151, 2630.68389691,  2755.09978424,  2883.11965784,  3019.47427305, 3162.27766017,  3311.83480822,  3468.4651304 ,  3629.6326025 , 3801.2928926 ,  3981.07170553,  4169.35299973,  4366.53889259, 4569.43671876,  4785.54422016,  5011.87233627,  5248.90444211, 5497.14677346,  5752.58000284,  6024.64322803,  6309.5734448 , 6607.97918625,  6920.49776548,  7242.06914895,  7584.57645675, 7943.28234724,  8318.95291817,  8712.39049922,  9117.22488558, 9548.4160391 , 10000.        , 10472.94122821, 10968.24979695]
lala = zeros(120)
get_subband_frequencies!(lala, f, settings)

In [14]:
lala-f_sb

120-element Array{Float64,1}:
 -0.007953288347756882
 -0.008329428049030696
 -0.008723359999997626
 -0.009135925458394922
 -0.009567997502294645
 -0.005228226862101337
 -0.005475489291818292
 -0.005734449999998503
 -0.006005651246653088
 -0.006289681066050434
 -0.002574223820829502
 -0.002695971395610286
 -0.002823469999995609
  ⋮
 -0.0036077246577406186
 -0.0037783481802762253
 -0.002140035287993669
 -0.0022412404669012176
 -0.0023472399998354376
 -0.0024582513724453747
 -0.002574511272541713
  2.166416379623115e-9
  4.1636667447164655e-9
  0.0
 -3.7325662560760975e-9
 -3.741661203093827e-9

In [ ]:
# x = [V_j_star, M_j, A_j_star, Tt_j_star, c_0, T_0, M_0, TS, theta]

In [22]:
l_i = 16  # Starting no. of the frequency band [-]



24-element Array{Float64,1}:
    50.11872336272725
    63.095734448019364
    79.43282347242817
   100.0
   125.89254117941675
   158.48931924611142
   199.52623149688807
   251.18864315095823
   316.22776601683796
   398.1071705534973
   501.18723362727246
   630.9573444801937
   794.3282347242822
  1000.0
  1258.9254117941687
  1584.893192461114
  1995.262314968881
  2511.886431509582
  3162.2776601683795
  3981.0717055349774
  5011.872336272725
  6309.573444801937
  7943.282347242822
 10000.0

In [18]:
for i in 
    println(i)
end

17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
